安裝依賴項目

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install ultralytics

驗證環境

In [ ]:
import sys, pkgutil
print("Python:", sys.executable)
print("ipykernel ok:", pkgutil.find_loader("ipykernel") is not None)
try:
    import torch
    print("Torch:", torch.__version__)
    print("CUDA available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print(torch.cuda.get_device_name(0))
except Exception as e:
    print("Torch import error:", e)


驗證YOLO ultralytics 套件

In [ ]:
import ultralytics
ultralytics.checks()

解壓並移動檔案，切分訓練、驗證、與測試數量

這邊因省時間"train"與"val"資料夾中只保留存在標註檔的圖片，只有"test"保留完整的

In [ ]:
import os, zipfile, shutil

def unzip_if_needed(zip_path, dest_dir):
    if os.path.isdir(dest_dir):
        return
    if os.path.exists(zip_path):
        os.makedirs(dest_dir, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(dest_dir)

def find_patient_root(root):
    for dirpath, dirnames, _ in os.walk(root):
        if any(d.startswith("patient") for d in dirnames):
            return dirpath
    return root

# 解壓縮（不使用外部 unzip 指令）
unzip_if_needed("training_image.zip", "./training_image")
unzip_if_needed("training_label.zip", "./training_label")

IMG_ROOT = find_patient_root("./training_image")
LBL_ROOT = find_patient_root("./training_label")

# 建立輸出資料夾
for p in ["./datasets/train/images","./datasets/train/labels",
          "./datasets/val/images","./datasets/val/labels",
          "./datasets/test/images","./datasets/test/labels"]:
    os.makedirs(p, exist_ok=True)

def move_patients(start, end, split):
    # train/val 僅搬移有標註的影像，test 保留全部影像
    moved = 0
    for i in range(start, end + 1):
        patient = f"patient{i:04d}"
        img_dir = os.path.join(IMG_ROOT, patient)
        lbl_dir = os.path.join(LBL_ROOT, patient)
        if not os.path.isdir(img_dir):
            continue

        if split == "test":
            label_lookup = {}
            if os.path.isdir(lbl_dir):
                label_lookup = {
                    os.path.splitext(fname)[0]: os.path.join(lbl_dir, fname)
                    for fname in os.listdir(lbl_dir)
                    if fname.endswith(".txt")
                }
            for fname in os.listdir(img_dir):
                if not fname.lower().endswith(".png"):
                    continue
                base = os.path.splitext(fname)[0]
                img_path = os.path.join(img_dir, fname)
                dst_img = f"./datasets/{split}/images/{base}.png"
                dst_lbl = f"./datasets/{split}/labels/{base}.txt"
                if os.path.exists(dst_img):
                    os.remove(dst_img)
                shutil.move(img_path, dst_img)
                lbl_path = label_lookup.get(base)
                if lbl_path and os.path.exists(lbl_path):
                    if os.path.exists(dst_lbl):
                        os.remove(dst_lbl)
                    shutil.move(lbl_path, dst_lbl)
                moved += 1
        else:
            if not os.path.isdir(lbl_dir):
                continue
            for lbl_name in os.listdir(lbl_dir):
                if not lbl_name.endswith(".txt"):
                    continue
                base = os.path.splitext(lbl_name)[0]
                img_path = os.path.join(img_dir, base + ".png")
                lbl_path = os.path.join(lbl_dir, lbl_name)
                if not os.path.exists(img_path):
                    continue
                dst_img = f"./datasets/{split}/images/{base}.png"
                dst_lbl = f"./datasets/{split}/labels/{base}.txt"
                if os.path.exists(dst_img):
                    os.remove(dst_img)
                shutil.move(img_path, dst_img)
                if os.path.exists(dst_lbl):
                    os.remove(dst_lbl)
                shutil.move(lbl_path, dst_lbl)
                moved += 1
    return moved

n_train = move_patients(1, 40, "train")
n_val   = move_patients(41, 45, "val")
n_test  = move_patients(46, 50, "test")

print(f"完成移動：train {n_train} 筆，val {n_val} 筆, test {n_test} 筆")

驗證圖片數量

In [ ]:
print('訓練集圖片數量 : ',len(os.listdir("./datasets/train/images")))
print('訓練集標記數量 : ',len(os.listdir("./datasets/train/labels")))
print('驗證集圖片數量 : ',len(os.listdir("./datasets/val/images")))
print('驗證集標記數量 : ',len(os.listdir("./datasets/val/labels")))
print('測試集圖片數量 : ',len(os.listdir("./datasets/test/images")))
print('測試集標記數量 : ',len(os.listdir("./datasets/test/labels")))
print('總圖片數量: ',len(os.listdir("./datasets/train/images")) + len(os.listdir("./datasets/val/images")) + len(os.listdir("./datasets/test/images")))
print('總標記數量: ',len(os.listdir("./datasets/train/labels")) + len(os.listdir("./datasets/val/labels")) + len(os.listdir("./datasets/test/labels")))

產生我們需要的.yaml檔案

In [ ]:
from textwrap import dedent
from pathlib import Path

data_yaml = dedent(
    """
train: "./datasets/train/images"
val: "./datasets/val/images"
test: "./datasets/test/images"

names:
  0: aortic_valve
"""
).strip()

config_path = Path("aortic_valve_colab.yaml")
config_path.write_text(data_yaml + "\n", encoding="utf-8")

print(f"YAML saved to: {config_path.resolve()}")
print(data_yaml)


開始訓練

若擔心GPU 記憶體不足可以嘗試降低batch

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo12n.pt') #初次訓練使用YOLO官方的預訓練模型，如要使用自己的模型訓練可以將'yolo12n.pt'替換掉
results = model.train(data="./aortic_valve_colab.yaml",
            epochs=10, #跑幾個epoch，這邊設定10做範例測試，可依需求調整
            batch=16, #batch_size
            imgsz=640, #圖片大小640*640
            device=0 #使用GPU進行訓練
            )

解壓縮testing_imgae

In [ ]:
import os, zipfile, shutil

def unzip_if_needed(zip_path, dest_dir):
    if os.path.isdir(dest_dir):
        return
    if os.path.exists(zip_path):
        os.makedirs(dest_dir, exist_ok=True)
        with zipfile.ZipFile(zip_path, 'r') as zf:
            zf.extractall(dest_dir)

def find_patient_root(root):
    for dirpath, dirnames, _ in os.walk(root):
        if any(d.lower().startswith("patient") for d in dirnames):
            return dirpath
    return root

# 解壓縮（不使用外部 unzip 指令）
unzip_if_needed("testing_image.zip", "./testing_image")

TEST_ROOT = find_patient_root("./testing_image")

# 建立輸出資料夾
DST_TEST = "./datasets/predict/images"
os.makedirs(DST_TEST, exist_ok=True)

# 收集所有圖片路徑（只看直屬的 patient 資料夾）
all_files = []
for patient_folder in os.listdir(TEST_ROOT):
    patient_path = os.path.join(TEST_ROOT, patient_folder)
    if os.path.isdir(patient_path) and patient_folder.lower().startswith("patient"):
        for fname in os.listdir(patient_path):
            if fname.lower().endswith(".png"):
                all_files.append(os.path.join(patient_path, fname))

# 按名稱排序並複製
all_files.sort()
copied = 0
for f in all_files:
    dst = os.path.join(DST_TEST, os.path.basename(f))
    if os.path.exists(dst): os.remove(dst)  # 不想覆蓋就刪掉這行
    shutil.copy2(f, dst)
    copied += 1

print(f"來源根目錄：{TEST_ROOT}")
print(f"完成複製！總共 {copied} 張，全部到 {DST_TEST}")


預測競賽分數約10分鐘

In [ ]:
from ultralytics import YOLO

model = YOLO(r'.\runs\detect\train\weights\best.pt') #請自行更改最新的best.pt檔路徑
results = model.predict(source="./datasets/predict/images/",
              save=True,
              imgsz=640,
              device=0
              )

預測數量

In [ ]:
print(len(results))

將偵測框數值寫進.txt檔

In [ ]:
os.makedirs('./predict_txt', exist_ok=True)


output_file = open('./predict_txt/images.txt', 'w', encoding='utf-8')

for i in range(len(results)):
    filename = str(results[i].path).replace('\\', '/').split('/')[-1].split('.png')[0]
    boxes = results[i].boxes
    box_num = len(boxes.cls.tolist())
    if box_num > 0:
        for j in range(box_num):
            label = int(boxes.cls[j].item())
            conf = boxes.conf[j].item()
            x1, y1, x2, y2 = boxes.xyxy[j].tolist()
            line = f"{filename} {label} {conf:.4f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}\n"
            output_file.write(line)
output_file.close()


釋放記憶體

In [ ]:
import torch ,gc

# 刪除大型變數
del boxes,all_files,results
gc.collect()
torch.cuda.empty_cache()

驗證模型(本周作業報告) 利用 YOLO 輸出預測結果

In [ ]:
!yolo detect val model=runs\detect\train\weights\best.pt data=aortic_valve_colab.yaml split=test save_txt save_conf